In [2]:
import numpy as np
import math
import pandas as pd
import itertools
import random

In [3]:
#GENERATE X, y
minx = 0
miny = 0
maxx = 10
maxy = 10
N = 10
random.seed(0)
rndy = [round(random.random()*(maxy - miny) + miny,2) for i in range(N)]
rndy = [0] + rndy
rndx = [round(random.random()*(maxx - minx) + minx,2) for i in range(N)]
rndx = [0] + rndx
df = pd.DataFrame()
df['i'] = [i for i in range(N + 1)]
df['x'] = rndx
df['y'] = rndy
df.to_excel('xyS3.xlsx', sheet_name='delta', index=None)


In [4]:
N = 10
M = 3
S = 5
T = 3
mindelta = 10
maxdelta = 30
minDemand = 50
maxDemand = 100
random.seed(0)

In [5]:
rnd_minDelta = [round(random.random()*mindelta,2)  for i in range(S*T*N)]
rnd_maxDelta = [round(random.random()*maxdelta, 2) for i in range(S*T*N)]
rndDelta = [round(random.random()*(rnd_maxDelta[i] - rnd_minDelta[i]) + rnd_minDelta[i],2) for i in range(S*T*N)]
rndDemand = [round(random.random()*(maxDemand - minDemand) + minDemand,2) for i in range(N)]
indices = [(i, t, s) for (i,t,s) in itertools.product(range(1, N+1), range(1, T+1), range(1, S+1)) ]
df = pd.DataFrame()
dfDemand = pd.DataFrame()

In [6]:
df['j'] = [i[0] for i in indices]
df['t'] = [i[1] for i  in indices]
df['s'] = [i[2] for i in indices]
df['delta'] = rndDelta
dfDemand['d'] = rndDemand

In [11]:
df.to_excel('scenarioS4.xlsx', sheet_name='delta', index=None)
dfDemand.to_excel('demandS4.xlsx', sheet_name='demand', index=None)

In [11]:
dfmean = df.groupby(['i', 't']).agg({'delta' : 'mean'})
dfmean = dfmean.reset_index()
dfmean

,i,t,delta
0,1,1,16.592
1,1,2,7.412
2,1,3,9.196
3,2,1,9.588
4,2,2,7.110
5,2,3,14.834
6,3,1,9.172
7,3,2,6.136
8,3,3,10.348
9,4,1,12.842


In [12]:
dfmean.to_excel('scenario_mean_S4.xlsx', sheet_name='delta', index=None)